# An example of using EASYMORE with source netCDF files in regular Latitude and Longitude for check of remapping file creation and remapping netcdf files
### merit hydro dataset is exracted from https://www.reachhydro.org/home/params/merit-basins

In [12]:

#
import geopandas as gpd
import glob
import os
import xarray as xr
import shutil

path_of_mertit_basin = '../../MERIT_Hydro/cat/' # were merit catchment is located;
# downloaded from # https://www.reachhydro.org/home/params/merit-basins

# paramaeters
path_efficiency = '../../ERA5_Efficiency_test/'
elements = [1,10,100,1000,10000,100000]
x_center, y_center = -103.140004, 45.461352

# create the folder
if os.path.isdir(path_efficiency):
    shutil.rmtree(path_efficiency)
os.makedirs(path_efficiency)

# merging the north american domain cat
Pfafs  = ['71', '72', '73', '74', '75', '76', '77', '78',
          '81', '82', '83', '84', '85', '86']

shp = None
for Pfaf in Pfafs:
    print(Pfaf)
    shp_temp = gpd.read_file(path_of_mertit_basin+'cat_pfaf_'+Pfaf+'_MERIT_Hydro_v07_Basins_v01_bugfix1.shp')
    if shp is None:
        shp = shp_temp
    else:
        shp = shp.append(shp_temp)

# shp.to_file(path_efficiency+'merged_NA.shp')

# matching the shapefile domain to netcdf domain to exclude shape with NaN that reduce easymore speed for 
# a fair compariosn across the number of elements
shp['lat'] = shp.centroid.y
shp['lon'] = shp.centroid.x
shp = shp [(shp['lat']<70) & (shp['lat']>10) & (shp['lon']<-54) & (shp['lon']>-165)]

# create the points from the centroid of the shapes in the shapefile

shp['dist'] = ((shp['lat']-y_center)**2.0 + (shp['lon']-x_center)**2.0)**0.5
shp_sorted = shp.sort_values(by='dist').copy()

shp

for element in elements:
    # when the shapes are randomly selected
    shp_temp = shp.sample(n=element)
    shp_temp.crs='epsg:4326'
    shp_temp.to_file(path_efficiency+str(element)+'_random.shp')
    # get the boundary, add buffer and subset the netcdf file
    shp_temp = shp_sorted.head(element)
    shp_temp.crs='epsg:4326'
    shp_temp.to_file(path_efficiency+str(element)+'_radial.shp')
    minx, miny, maxx, maxy = shp_temp.total_bounds
    minx, miny, maxx, maxy = minx-1, miny-1, maxx+1, maxy+1 # add buffer
    nc_names = glob.glob('./data/Source_nc_ERA5/ERA5_NA_1979*.nc')
    for nc_name in nc_names:
        nc_name_base = os.path.basename(nc_name)
        ds = xr.open_dataset(nc_name)
        ds_subset = ds.sel(latitude=slice(maxy, miny), longitude=slice(minx, maxx))
        # save
        if os.path.isfile(path_efficiency+nc_name_base+'_'+str(element)):
            os.remove(path_efficiency+nc_name_base+'_'+str(element))
        ds_subset['airtemp'].encoding['_FillValue'] = None
        ds_subset['pptrate'].encoding['_FillValue'] = None
        ds_subset.to_netcdf(path_efficiency+nc_name_base+'_'+str(element))




71
72
73
74
75
76
77
78
81
82
83
84
85
86


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiPolygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]


In [11]:
# loading EASYMORE
from easymore import Easymore

for element in elements:
    
    # initializing EASYMORE object
    esmr = Easymore()

    # specifying EASYMORE objects
    # name of the case; the temporary, remapping and remapped file names include case name
    esmr.case_name                = 'Merit_random_'+ str(element)            
    # temporary path that the EASYMORE generated GIS files and remapped file will be saved
    esmr.temp_dir                 = './temporary/'
    # name of target shapefile that the source netcdf files should be remapped to
    esmr.target_shp               = path_efficiency+str(element)+'_random.shp'
    esmr.target_shp_ID            = 'COMID' # if not provided easymore give ID according to shape order in shapefile
    # name of netCDF file(s); multiple files can be specified with *
    esmr.source_nc                = './data/Source_nc_ERA5/ERA5_NA_*.nc'
    # name of variables from source netCDF file(s) to be remapped
    esmr.var_names                = ['airtemp','pptrate']
    # rename the variables from source netCDF file(s) in the remapped files;
    # it will be the same as source if not provided
    esmr.var_names_remapped       = ['temperature','precipitation']
    # name of variable longitude in source netCDF files
    esmr.var_lon                  = 'longitude'
    # name of variable latitude in source netCDF files
    esmr.var_lat                  = 'latitude'
    # name of variable time in source netCDF file; should be always time
    esmr.var_time                 = 'time'
    # location where the remapped netCDF file will be saved
    esmr.output_dir               = './output/'

    # execute EASYMORE
    esmr.nc_remapper()


EASYMORE version 0.0.5 is initiated.
no author name is provided. The author name is changed to (author name)!
EASYMORE is given multiple variables for remapping but only on format and fill value. EASYMORE repeats the format and fill value for all the variables in output files
EASYMORE will remap variable  airtemp  from source file to variable  temperature  in remapped netCDF file
EASYMORE will remap variable  pptrate  from source file to variable  precipitation  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude and longitude
EASYMORE detects that all the variables have dimensions of:
['time', 'latitude', 'longitude']
EASYMORE detects that the longitude variables has dimensions of:
['longitude']
EASYMORE detects that the latitude variables has dimensions of:
['latitude']
--CREATING-REMAPPING-FILE--
Started at date and time 2023-03-31 21:28:44.039681
EASYMORE detects that target shapefile is in WGS84

/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiPolygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]


max difference of lat values in source nc files are :  0.0
max difference of lon values in source nc files are :  0.0


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1638: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  result[:] = values
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]


./temporary/Merit1_source_shapefile.shp
EASYMORE created the shapefile from the netCDF file and saved it here:
EASMORE detects that target shapefile is inside the boundary of source netCDF file  and therefore correction for longitude values -180 to 180 or 0 to 360 is not performed even if  the correction_shp_lon flag is set to True [default is True]


...

it seems everything is OK with the sink/target shapefile; added to EASYMORE object target_shp_gpd
EASYMORE saved target shapefile for EASYMORE claculation as:
./temporary/Merit100000_target_shapefile.shp
EASYMORE detects case 1 - regular lat/lon
max difference of lat values in source nc files are :  0.0
max difference of lon values in source nc files are :  0.0


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1638: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  result[:] = values


./temporary/Merit100000_source_shapefile.shp
EASYMORE created the shapefile from the netCDF file and saved it here:
EASMORE detects that target shapefile is inside the boundary of source netCDF file  and therefore correction for longitude values -180 to 180 or 0 to 360 is not performed even if  the correction_shp_lon flag is set to True [default is True]


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/easymore/easymore.py:216: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiPolygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon

Ended at date and time 2023-03-31 21:56:47.653481
It took 1114.663977 seconds to finish creating of the remapping file
---------------------------
------REMAPPING------
netcdf output file will be compressed at level 4
Removing existing remapped .nc file.
Remapping ./data/Source_nc_ERA5/ERA5_NA_19790101.nc to ./output/Merit100000_remapped_1979-01-01-00-00-00.nc 
Started at date and time 2023-03-31 21:56:48.307555 
Ended at date and time 2023-03-31 21:56:52.057976 
It took 3.750421 seconds to finish the remapping of variable(s) 
---------------------
Removing existing remapped .nc file.
Remapping ./data/Source_nc_ERA5/ERA5_NA_19790102.nc to ./output/Merit100000_remapped_1979-01-02-00-00-00.nc 
Started at date and time 2023-03-31 21:56:52.069707 
Ended at date and time 2023-03-31 21:56:55.940315 
It took 3.870608 seconds to finish the remapping of variable(s) 
---------------------
Removing existing remapped .nc file.
Remapping ./data/Source_nc_ERA5/ERA5_NA_19790103.nc to ./output/Merit10

In [2]:
# loading EASYMORE
from easymore import Easymore

for element in elements:
    
    # initializing EASYMORE object
    esmr = Easymore()

    # specifying EASYMORE objects
    # name of the case; the temporary, remapping and remapped file names include case name
    esmr.case_name                = 'Merit_radial_'+ str(element)            
    # temporary path that the EASYMORE generated GIS files and remapped file will be saved
    esmr.temp_dir                 = './temporary/'
    # name of target shapefile that the source netcdf files should be remapped to
    esmr.target_shp               = path_efficiency+str(element)+'_radial.shp'
    esmr.target_shp_ID            = 'COMID' # if not provided easymore give ID according to shape order in shapefile
    # name of netCDF file(s); multiple files can be specified with *
    esmr.source_nc                = path_efficiency+'ERA5_NA_*.nc_'+str(element)
    # name of variables from source netCDF file(s) to be remapped
    esmr.var_names                = ['airtemp','pptrate']
    # rename the variables from source netCDF file(s) in the remapped files;
    # it will be the same as source if not provided
    esmr.var_names_remapped       = ['temperature','precipitation']
    # name of variable longitude in source netCDF files
    esmr.var_lon                  = 'longitude'
    # name of variable latitude in source netCDF files
    esmr.var_lat                  = 'latitude'
    # name of variable time in source netCDF file; should be always time
    esmr.var_time                 = 'time'
    # location where the remapped netCDF file will be saved
    esmr.output_dir               = './output/'

    # execute EASYMORE
    esmr.nc_remapper()

EASYMORE version 0.0.5 is initiated.
no author name is provided. The author name is changed to (author name)!
EASYMORE is given multiple variables for remapping but only on format and fill value. EASYMORE repeats the format and fill value for all the variables in output files
EASYMORE will remap variable  airtemp  from source file to variable  temperature  in remapped netCDF file
EASYMORE will remap variable  pptrate  from source file to variable  precipitation  in remapped netCDF file
EASYMORE detects that the variables from the netCDF files are identicalin dimensions of the variables and latitude and longitude
EASYMORE detects that all the variables have dimensions of:
['time', 'latitude', 'longitude']
EASYMORE detects that the longitude variables has dimensions of:
['longitude']
EASYMORE detects that the latitude variables has dimensions of:
['latitude']


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


--CREATING-REMAPPING-FILE--
Started at date and time 2023-04-05 21:03:21.463236
EASYMORE detects that target shapefile is in WGS84 (epsg:4326)
EASYMORE detects that the field for ID is provided in sink/target shapefile
EASYMORE detects that either of the fields for latitude or longitude is not provided in sink/target shapefile
calculating centroid of shapes in WGS84 projection;
for better appximation use the easymore equal area centroid function to preprocess target shapefile
EASYMORE detects that no field for latitude is provided in sink/target shapefile
latitude values are added in the field lat_t
EASYMORE detects that no field for longitude is provided in sink/target shapefile
longitude values are added in the field lon_t
it seems everything is OK with the sink/target shapefile; added to EASYMORE object target_shp_gpd
EASYMORE saved target shapefile for EASYMORE claculation as:
./temporary/Merit_radial_1_target_shapefile.shp
EASYMORE detects case 1 - regular lat/lon
max difference o

/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1638: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  result[:] = values
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/easymore/easymore.py:216: 

Ended at date and time 2023-04-05 21:03:22.046706
It took 0.58347 seconds to finish creating of the remapping file
---------------------------
------REMAPPING------
netcdf output file will be compressed at level 4
Removing existing remapped .nc file.
Remapping ../../ERA5_Efficiency_test/ERA5_NA_19790101.nc_1 to ./output/Merit_radial_1_remapped_1979-01-01-00-00-00.nc 
Started at date and time 2023-04-05 21:03:22.063571 
Ended at date and time 2023-04-05 21:03:22.639406 
It took 0.575835 seconds to finish the remapping of variable(s) 
---------------------
Removing existing remapped .nc file.
Remapping ../../ERA5_Efficiency_test/ERA5_NA_19790102.nc_1 to ./output/Merit_radial_1_remapped_1979-01-02-00-00-00.nc 
Started at date and time 2023-04-05 21:03:22.643411 
Ended at date and time 2023-04-05 21:03:22.885625 
It took 0.242214 seconds to finish the remapping of variable(s) 
---------------------
Removing existing remapped .nc file.
Remapping ../../ERA5_Efficiency_test/ERA5_NA_19790103.n

...

it seems everything is OK with the sink/target shapefile; added to EASYMORE object target_shp_gpd
EASYMORE saved target shapefile for EASYMORE claculation as:
./temporary/Merit_radial_100000_target_shapefile.shp
EASYMORE detects case 1 - regular lat/lon
max difference of lat values in source nc files are :  0.0
max difference of lon values in source nc files are :  0.0


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:1638: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  result[:] = values


./temporary/Merit_radial_100000_source_shapefile.shp
EASYMORE created the shapefile from the netCDF file and saved it here:
EASMORE detects that target shapefile is inside the boundary of source netCDF file  and therefore correction for longitude values -180 to 180 or 0 to 360 is not performed even if  the correction_shp_lon flag is set to True [default is True]


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/easymore/easymore.py:216: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  shp_int.to_file(self.temp_dir+self.case_name+'_intersected_shapefile.shp') # save the intersected files
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]
/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiPolygon

Ended at date and time 2023-04-05 21:19:24.492692
It took 864.545048 seconds to finish creating of the remapping file
---------------------------
------REMAPPING------
netcdf output file will be compressed at level 4
Remapping ../../ERA5_Efficiency_test/ERA5_NA_19790101.nc_100000 to ./output/Merit_radial_100000_remapped_1979-01-01-00-00-00.nc 
Started at date and time 2023-04-05 21:19:25.314542 
Ended at date and time 2023-04-05 21:19:29.006303 
It took 3.691761 seconds to finish the remapping of variable(s) 
---------------------
Remapping ../../ERA5_Efficiency_test/ERA5_NA_19790102.nc_100000 to ./output/Merit_radial_100000_remapped_1979-01-02-00-00-00.nc 
Started at date and time 2023-04-05 21:19:29.014205 
Ended at date and time 2023-04-05 21:19:33.051686 
It took 4.037481 seconds to finish the remapping of variable(s) 
---------------------
Remapping ../../ERA5_Efficiency_test/ERA5_NA_19790103.nc_100000 to ./output/Merit_radial_100000_remapped_1979-01-03-00-00-00.nc 
Started at dat